## Code is for analysis of CMIP6 model simulations

In [ ]:
## Import required modules
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import dask
import glob 

In [ ]:
#define study area , here it is India
IND = 'India'
IND_ne_lat = 45
IND_ne_lon = 100
IND_sw_lat = 6
IND_sw_lon = 66

In [ ]:
#uploading files
file_list = glob.glob("\path\*.nc")

In [ ]:
# subsetting over required domain and coverting unit 
for file in file_list:
    data = xr.open_dataset(file)['gpp']
    data_subset = data.sel(lat = slice(IND_sw_lat, IND_ne_lat), lon = slice(IND_sw_lon, IND_ne_lon))
    data_subset['gpp_c']= data_subset*1000*24*60*60
    data_subset.to_dataset(name = 'gpp_new').to_netcdf('path\'+'india_'+file[56:],mode='w',format='NETCDF4')
    

In [ ]:
#files then regridded in CDO 

In [ ]:
#masking using shape file

In [ ]:
#import required modules
import geopandas as gpd
import cartopy.feature as cf
from matplotlib.pyplot import cm
from matplotlib.collections import LineCollection
from osgeo import gdal,osr,ogr

In [ ]:
#read regridded files
file_list = glob.glob('\path\*.nc')


In [ ]:
#Define mask function (https://mygeoblog.com/2019/06/25/mask-netcdf-using-shp-file/)

def makeMask(lon,lat,res):
    source_ds = ogr.Open('\path\india_st.shp')
    source_layer = source_ds.GetLayer()

    # Create high res raster in memory
    mem_ds = gdal.GetDriverByName('MEM').Create('', lon.size, lat.size, gdal.GDT_Byte)
    mem_ds.SetGeoTransform((lon.min(), res, 0, lat.max(), 0, -res))
    band = mem_ds.GetRasterBand(1)

    # Rasterize shapefile to grid
    gdal.RasterizeLayer(mem_ds, [1], source_layer, burn_values=[1])

    # Get rasterized shapefile as numpy array
    array = band.ReadAsArray()

    # Flush memory file
    mem_ds = None
    band = None
    return array


In [ ]:
#mask each file and save 

for file in file_list:
    data = xr.open_dataset(file)
    name =file[86:97]
    gpp=data.gpp_c
    lon=data.lon
    lat=data.lat
    cellsize=lon[:][1] - lon[:][0]
    mask = makeMask(lon,lat[::-1],cellsize)
    mask_gpp = xr.zeros_like(gpp)
    for ind, val in enumerate(gpp['time']):
      gpp_new = gpp[ind]
      gppdata= np.ma.masked_where(mask==0,gpp_new)
      mask_gpp[ind] = gppdata
    mask_gpp.to_netcdf('\path\'+file[86:97]+'masked.nc',mode='w',format='NETCDF4')



In [ ]:
#read masked files
file_list = glob.glob('\path\*masked.nc')
modis_gpp=xr.open_dataset("\path\MOD17A2HGF.061_500m_filtered_1by1_merged_masked.nc") 


In [ ]:
#plotting

fig,ax = plt.subplots(figsize=(20,9))
plt.subplots_adjust( bottom=0.2)
color=iter(cm.Set1(np.linspace(0,1,7)))
labels = ["ACCESS-ESM1-5","NorESM2-LM","MODIS"]

for file in file_list:
    data= xr.open_dataset(file)
    
    name =file[64:74]
    # print(data)
    c=next(color)
    gpp_mean=(data['gpp_c'].mean('lat').mean('lon'))*30 #spatial mean and unit converted daily to monthly
 
    l1=plt.plot(gpp_mean['time'],gpp_mean, label=name, linewidth=1.5,linestyle='--',marker=".",c=c)

    plt.yticks(fontsize=15)
    plt.xticks(fontsize=15)
    plt.tick_params(axis='x',labelsize=12)
    plt.tick_params(axis='y',labelsize=12)
    l2=l1
ax.set_xlim([dt.date(2000, 1, 15), dt.date(2014, 1, 16)])
plt.xlabel('Time (Year)', fontsize=18,labelpad=15)
plt.ylabel('GPP (gC m$^{-2}$ mo$^{-1}$)', fontsize=18,labelpad=10)


monthly_data = modis_gpp['Gpp_day'].resample(time='1M').mean() 
mean=(monthly_data.mean('lat').mean('lon'))*30

 
l3=plt.plot(mean['time'],mean,linestyle='--',marker=".",c='Green')
plt.legend([l1,l2,l3],labels=labels,loc='upper center', bbox_to_anchor=(0.5, 1.15),ncol=4,fontsize=15)
# plt.savefig('\path\'+'figure_2_Models_Modis_past_gpp.png',dpi=600)

plt.show()


In [ ]:
#making dataframe 
access=xr.open_dataset('\path\ACCESS-ESM1past_masked.nc')
nor=xr.open_dataset('\path\NorESM2-LM_past_masked.nc')
modis=xr.open_dataset("\path\MOD17A2HGF.061_500m_filtered_1by1_merged_masked.nc") 

months = ["January","February","March","April","May","June","July","August","September","October","November","December"]

access=access.sel(time=slice('2000-01-16T12:00:00','2014-12-16T12:00:00.000000000')).mean('lat').mean('lon')
access=access.to_dataframe()
access = access.reset_index(level=0)
access['month']=pd.to_datetime(access['time']).dt.month
access.index = access['time']

nor=nor.sel(time=slice('2000-01-16T12:00:00','2014-12-16T12:00:00.000000000')).mean('lat').mean('lon')
nor=nor.to_dataframe()
nor = nor.reset_index(level=0)
nor['month']=pd.to_datetime(nor['time']).dt.month
nor.index = nor['time']

modis=modis.sel(time=slice('2000-01-16T12:00:00','2014-12-16T12:00:00.000000000')).mean('lat').mean('lon')
modis=modis.to_dataframe()
modis = modis.reset_index(level=0)
modis['month']=pd.to_datetime(modis['time']).dt.month
modis.index = modis['time']

data=access.groupby(access['month']).mean()*30
data2=access.groupby(access['month']).std()*30

data4=nor.groupby(nor['month']).mean()*30
data5=nor.groupby(nor['month']).std()*30

data_m=modis.groupby(modis['month']).mean()*30
data2_m=modis.groupby(modis['month']).std()*30


data['mean_gpp_access']= data['gpp_c']
data=data[['mean_gpp_access']]
data2['std_gpp_access']=data2['gpp_c']
data2=data2[['std_gpp_access']]


data4['mean_gpp_nor']= data4['gpp_c']
data4=data4[['mean_gpp_nor']]
data5['std_gpp_nor']=data5['gpp_c']
data5=data5[['std_gpp_nor']]


data_m['mean_gpp_modis']= data_m['Gpp_day']
data_m=data_m[['mean_gpp_modis']]
data2_m['std_gpp_modis']=data2_m['Gpp_day']
data2_m=data2_m[['std_gpp_modis']]

data.reset_index(level=0, inplace=True)
data2.reset_index(level=0, inplace=True)
data_m.reset_index(level=0, inplace=True)
data2_m.reset_index(level=0, inplace=True)
data4.reset_index(level=0, inplace=True)
data5.reset_index(level=0, inplace=True)
data7 = pd.merge(data, data2, on='month').merge(data4, on='month').merge(data5, on='month').merge(data_m, on='month').merge(data2_m ,on='month')


#plotting errorbar 

labels = ["ACCESS-ESM1-5","NorESM2-LM","MODIS"]
fig,ax = plt.subplots(figsize=(20,12))
color=iter(cm.Set1(np.linspace(0,1,7)))
c=next(color)
l1=plt.plot(data7['month'], data7['mean_gpp_access'],linewidth=2,linestyle='--',marker="o",c=c)

plt.errorbar(data7['month'], data7['mean_gpp_access'],
             yerr = data7['std_gpp_access'],
             fmt ='o',c=c,capsize=8)
c=next(color)
l2=plt.plot(data7['month'], data7['mean_gpp_nor'],linewidth=2,linestyle='--',marker="o",c=c)

plt.errorbar(data7['month'], data7['mean_gpp_nor'],
             yerr = data7['std_gpp_nor'],
             fmt ='o',c=c,capsize=8)

c=next(color)
l3=plt.plot(data7['month'], data7['mean_gpp_modis'],linewidth=2,linestyle='--',marker="o",c='Green')

plt.errorbar(data7['month'], data7['mean_gpp_modis'],
             yerr = data7['std_gpp_modis'],
             fmt ='o',c='Green',capsize=8)



plt.xticks(data7['month'],months)
plt.xlabel('Time (Month)', fontsize=18,labelpad=15)
plt.ylabel('GPP (gC m$^{-2}$ mo$^{-1}$)', fontsize=18,labelpad=10)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15),ncol=4,fontsize=15)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.legend([l1,l2,l3],labels=labels,loc='upper center', bbox_to_anchor=(0.5, 1.15),ncol=4,fontsize=15)

plt.grid('on', linestyle='--',alpha=0.4)
plt.tick_params(axis='x',labelsize=12)
plt.tick_params(axis='y',labelsize=12)

plt.show()


In [ ]:
dataset= xr.open_dataset('\path\MOD17A2HGF.061_500m_filtered_1by1_merged.nc')

In [ ]:
india= gpd.read_file("\path\india_st.shp")

polygon = india.geometry.unary_union

india_ex = gpd.GeoDataFrame(geometry=[polygon], crs=india.crs)
poly= india_ex['geometry'].values

In [ ]:
#spatial plots

fig,ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
lon=dataset['lon']
lat=dataset['lat']
time=dataset['time']
gpp_temporal_mean= dataset['Gpp_day'].mean('time')

cs=ax.contourf(lon[:],lat[:],gpp_temporal_mean[:,:],levels=np.arange(0,10,1),cmap="YlGn" )

ax.coastlines(alpha=0.4)
g=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True)
g.top_labels=False
g.right_labels=False
ax.add_geometries(poly, crs=ccrs.PlateCarree(), facecolor='none', edgecolor='0.2')


cbar_ax = fig.add_axes([0.8, 0.15, 0.02, 0.7])
fig.colorbar(cs, cax=cbar_ax,label='GPP (gC m$^{-2}$ d$^{-1}$)')

plt.show()

In [ ]:
#spatial trend analysis

d=nor_gpp.resample(time='1M').mean()  #resampling data month-wise
gpp_year = d.groupby('time.year').mean('time')['gpp_c']*365



In [ ]:
#define trend function
def trend(y_array,alpha=0.05):
    shape      =      [y_array.shape[1],y_array.shape[2]]
    if y_array[0,0,:].dims==('longitude',):
        mk_res     =      xr.DataArray(np.empty(shape), coords=[y_array['latitude'], y_array['longitude']], dims=['latitude','longitude'])
        trend_1     =      xr.DataArray(np.empty(shape), coords=[y_array['latitude'], y_array['longitude']], dims=['latitude','longitude'])
    else:
        mk_res     =      xr.DataArray(np.empty(shape), coords=[y_array['lat'], y_array['lon']], dims=['lat','lon'])
        trend_1     =      xr.DataArray(np.empty(shape), coords=[y_array['lat'], y_array['lon']], dims=['lat','lon'])


    for i in range(y_array.shape[1]):
        for j in range(y_array.shape[2]):
            temp              = y_array[:,i,j]
            if np.all(np.isnan(temp)):
                mk_res[i,j]= np.nan
                trend_1[i,j] =np.nan
            else:
                trend,h,p,z,t,s,var_s,slp,intr = mk.original_test(temp,alpha=alpha)
                h1 =h*1.0

                mk_res[i,j] = h1
                trend_1[i,j]=slp
        print(i)
    return trend_1, mk_res

In [ ]:
t,sig = trend(gpp_year)

In [ ]:
#plotting spatial trend 
fig = plt.figure(figsize=(10,9))
ax = plt.subplot(111,projection=ccrs.PlateCarree())
cs=ax.contourf(lon,lat[::-1],t,levels=np.arange(-4,4,0.2),cmap="RdBu",extend='both' )
g=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True)
g.top_labels=False
g.right_labels=False

ax.add_geometries(poly, crs=ccrs.PlateCarree(), facecolor='none', edgecolor='0.2')
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(cs, cax=cbar_ax,label='GPP (gC m$^{-2}$ y$^{-1}$)',extend='both')
cbar_ax.tick_params(labelsize=18)
plt.show()

In [ ]:
modis_gpp=modis_gpp.resample(time='1M').mean()  #resampling data month-wise


In [ ]:
#regional analysis

fig, axs = plt.subplots(figsize=(25,10))
labels = ["Western Himalaya","Eastern Himalaya","Betul Forest"] #3 selected regions

#subsetting over spacified region and taking spatial mean 
# converting unit from daily to monthly

data_west= modis_gpp.sel(lat=slice(30.5,29.5),lon=slice(78,79)).mean('lat').mean('lon')*30

data_east=modis_gpp.sel(lat=slice(26.5,25.5),lon=slice(93,94)).mean('lat').mean('lon')*30


data_B= modis_gpp.sel(lat=slice(22.5,21.5),lon=slice(77,78)).mean('lat').mean('lon')*30



l1=axs.plot(data_west['time'],data_west['Gpp_day'], label='WESTERN HIMALAYA', 
            linewidth=1.5,linestyle='-',marker=".",color="skyblue",markersize=3)

l2=axs.plot(data_east['time'],data_east['Gpp_day'], label='EASTERN HIMALAYA', 
            linewidth=1.5,linestyle='-',marker=".",color="orange",markersize=3)

l3=axs.plot(data_B['time'],data_B['Gpp_day'], label='BETUL', 
            linewidth=1.5,linestyle='-',marker=".",color="green",markersize=3)



axs.set_ylabel('MODIS-GPP (gC m$^{-2}$ mo$^{-1}$)', fontsize=15,labelpad=10)
axs.set_xlabel('Time (Year)', fontsize=15,labelpad=15)
axs.tick_params(axis='x',labelsize=12)
axs.tick_params(axis='y',labelsize=12)
axs.grid(linestyle='--')

fig.legend([l1, l2,l3],labels=labels,loc="upper center",ncol=4,fontsize=18)

plt.show() 